<a href="https://colab.research.google.com/github/sjdee/Market-Analysis-Techniques/blob/master/transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# infotech + 4'1p-pYZfGNprIIo1SL4kC90kz7JJXT5qaj'
# all '1ap1xR_SE5hJYK_Rzd1j72_sbrM8l-N1N'
# apple '1KPTQoFCDusB_ktMc6GyJbv6rEMOKF9lJ'
file_id = '1ap1xR_SE5hJYK_Rzd1j72_sbrM8l-N1N'


# 2. Load a file by ID 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('sp500_transformation_input.xlsx')

     |████████████████████████████████| 993kB 3.5MB/s 


In [0]:
import pandas as pd
import numpy as np
import time
from time import gmtime, strftime
from google.colab import files

print("Started at " + str(strftime("%H:%M:%S", gmtime())))

frames = []

xl = pd.ExcelFile('sp500_transformation_input.xlsx')


#sheets = 68 + 4
number_of_sheets = len(xl.sheet_names)

print("number of firms: " + str(number_of_sheets))


change_percent = 0.05


# iterate through the excel sheets
for k in range(number_of_sheets):
  
    start = time.time()
    
    # get firm data for sheet k
    df = pd.read_excel(xl, header=None, sheet_name=k)

    # extract firm name
    firm = df.iloc[0][1]

    # set new header
    new_header = df.iloc[2]
    df.columns = new_header
    df.columns.name = ''

    # delete top 3 rows
    df = df.iloc[3:,:]
    
    # insert column for firm name    
    df['FIRM'] = firm
    df.at[0,'Dates'] = "5/3/1999  12:00:00 am"
    
    
    df.dropna(subset=['PX_OFFICIAL_CLOSE'], inplace=True)
    # df.dropna(inplace=True)
    
    # reset index and begin numbering from 0 to compensate for the above delete     
    df.reset_index(inplace=True, drop=True)
    
    #get the number of rows in the sheet
    row_count = len(df.index)
    #   iterate through rows for a given firm   
    for i in range(0,row_count-10):
      
      
      anlayst_recs = df.iloc[i]['TOT_ANALYST_REC']
      
      if(anlayst_recs ==0 ):
        df.at[i,'hold_percent'] = 0
        df.at[i,'buy_percent']  = 0
        df.at[i,'sell_percent'] = 0
        
      else:
        df.at[i,'hold_percent'] = df.iloc[i]['TOT_HOLD_REC']/anlayst_recs
        df.at[i,'buy_percent'] = df.iloc[i]['TOT_BUY_REC']/anlayst_recs
        df.at[i,'sell_percent'] = df.iloc[i]['TOT_SELL_REC']/anlayst_recs

      # df.iloc[i:i+11]['PX_OFFICIAL_CLOSE']
      closeprice0 = df.iloc[i]['PX_OFFICIAL_CLOSE']      
      closeprice1 = df.iloc[i+1]['PX_OFFICIAL_CLOSE'] 
      closeprice2 = df.iloc[i+2]['PX_OFFICIAL_CLOSE'] 
      closeprice3 = df.iloc[i+3]['PX_OFFICIAL_CLOSE']
      closeprice4 = df.iloc[i+4]['PX_OFFICIAL_CLOSE'] 
      closeprice5 = df.iloc[i+5]['PX_OFFICIAL_CLOSE']
      closeprice6 = df.iloc[i+6]['PX_OFFICIAL_CLOSE'] 
      closeprice7 = df.iloc[i+7]['PX_OFFICIAL_CLOSE']
      closeprice8 = df.iloc[i+8]['PX_OFFICIAL_CLOSE'] 
      closeprice9 = df.iloc[i+9]['PX_OFFICIAL_CLOSE']
      closeprice10 = df.iloc[i+10]['PX_OFFICIAL_CLOSE']
      
      #   calculates the sample standard deviation rather than popuation std
      #   https://stackoverflow.com/questions/15389768/standard-deviation-of-a-list
      #   6th element (as the count begins from zero) takes in the std of 5 elements begining at index i
      #   df.iloc[i:i+5]['PX_OFFICIAL_CLOSE']
      df.at[i+5,'std_5day'] = np.std([closeprice0,closeprice1,closeprice2,closeprice3,closeprice4], ddof=1) 
      
      #   df.iloc[i:i+10]['PX_OFFICIAL_CLOSE']  
      #   11th element (as the count begins from zero) takes in the std of 10 elements begining at index i
      df.at[i+10,'std_10day'] = np.std([closeprice0,closeprice1,closeprice2,closeprice3,closeprice4,closeprice5,closeprice6,closeprice7,closeprice8,closeprice9], ddof=1) 
      
      

      #   change percentage over past 5 days
      df.at[i+5,'change_5day'] = (closeprice4-closeprice0)/closeprice0
      
      #   change percentage over past 10 days
      df.at[i+10,'change_10day'] =  (closeprice9-closeprice0)/closeprice0
      
      
      
      # day1 calculation     
      # buy=2 sell=0
      if(closeprice1>=closeprice0):
          df.at[i,'day1'] = 2 

      else:
          df.at[i,'day1'] = 0  
          
      # day2 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice2-closeprice0>= change_percent * closeprice0):
          df.at[i,'day2'] = 2

      elif(closeprice2-closeprice0<= change_percent * closeprice0):
          df.at[i,'day2'] = 0

      else:
          df.at[i,'day2']= 1

      # day3 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice3-closeprice0>= change_percent * closeprice0):
          df.at[i,'day3'] = 2

      elif(closeprice3-closeprice0<= change_percent * closeprice0):
          df.at[i,'day3'] = 0

      else:
          df.at[i,'day3']= 1
          
      # day4 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice4-closeprice0>= change_percent * closeprice0):
          df.at[i,'day4'] = 2 

      elif(closeprice4-closeprice0<= change_percent * closeprice0):
          df.at[i,'day4'] = 0 

      else:
          df.at[i,'day4']= 1

      # day5 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice5-closeprice0>= change_percent * closeprice0):
          df.at[i,'day5'] = 2 

      elif(closeprice5-closeprice0<= change_percent * closeprice0):
          df.at[i,'day5'] = 0 

      else:
          df.at[i,'day5']= 1
          
      # day6 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice6-closeprice0>= change_percent * closeprice0):
          df.at[i,'day6'] = 2 

      elif(closeprice6-closeprice0<= change_percent * closeprice0):
          df.at[i,'day6'] = 0 

      else:
          df.at[i,'day6']= 1

      # day7 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice7-closeprice0>= change_percent * closeprice0):
          df.at[i,'day7'] = 2 

      elif(closeprice7-closeprice0<= change_percent * closeprice0):
          df.at[i,'day7'] = 0 

      else:
          df.at[i,'day7']= 1
          
      # day8 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice8-closeprice0>= change_percent * closeprice0):
          df.at[i,'day8'] = 2 

      elif(closeprice8-closeprice0<= change_percent * closeprice0):
          df.at[i,'day8'] = 0 

      else:
          df.at[i,'day8']= 1

      # day9 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice9-closeprice0>= change_percent * closeprice0):
          df.at[i,'day9'] = 2 

      elif(closeprice9-closeprice0<= change_percent * closeprice0):
          df.at[i,'day9'] = 0 

      else:
          df.at[i,'day9']= 1

      # day10 calculation    
      # buy=2 sell=0 hold=1
      if(closeprice10-closeprice0>= change_percent * closeprice0):
          df.at[i,'day10'] = 2 

      elif(closeprice10-closeprice0<= change_percent * closeprice0):
          df.at[i,'day10'] = 0 

      else:
          df.at[i,'day10']= 1
    
    # append to the file
    frames.append(df)  
    end = time.time()
    print("firm number "+ str(k)+ " done and took " + str( round((end - start)/60,2) ) + " minutes")
    
#     if(k%20==0):
#       print("Saving next 20 firms up till",k)
#       result = pd.concat(frames)
#       filename = "firmsuptill" + str(k) + ".xlsx"
#       result.to_excel(filename)
#       frames[:] = []
      
result = pd.concat(frames)
print("Ended at " + str(strftime("%H:%M:%S", gmtime())))

Started at 22:10:26
number of firms: 505
firm number 0 done and took 0.11 minutes
firm number 1 done and took 0.1 minutes
firm number 2 done and took 0.09 minutes
firm number 3 done and took 0.09 minutes
firm number 4 done and took 0.09 minutes
firm number 5 done and took 0.09 minutes
firm number 6 done and took 0.09 minutes
firm number 7 done and took 0.09 minutes
firm number 8 done and took 0.09 minutes
firm number 9 done and took 0.09 minutes
firm number 10 done and took 0.09 minutes
firm number 11 done and took 0.09 minutes
firm number 12 done and took 0.09 minutes
firm number 13 done and took 0.09 minutes
firm number 14 done and took 0.09 minutes
firm number 15 done and took 0.09 minutes
firm number 16 done and took 0.1 minutes
firm number 17 done and took 0.09 minutes
firm number 18 done and took 0.09 minutes
firm number 19 done and took 0.09 minutes
firm number 20 done and took 0.09 minutes
firm number 21 done and took 0.09 minutes
firm number 22 done and took 0.09 minutes
firm 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:223: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Ended at 23:01:53


In [0]:
result = pd.concat(frames, sort=False)
result.to_excel("transform_output.xlsx")

In [0]:
from google.colab import files

files.download("transform_output.xlsx")

In [0]:
# from google.colab import files

# # Create & upload a file.
# uploaded = drive.CreateFile({'title': 'transform_output.xlsx'})
# uploaded.SetContentFile('transform_output.xlsx')
# uploaded.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))

# # result.to_csv(index=False)

In [0]:
# f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": fid}]})
# f.SetContentFile( some_path )
# f.Upload()